In [62]:
# https://www.ohmsha.co.jp/book/9784274226984/
# Generic Algorithm
%config IPCompleter.greedy=True
import sys
import numpy as np
import scipy as sp
import scipy.stats
import sympy as symp
import itertools

import matplotlib.pyplot as plt
import seaborn as sns

In [63]:
# > conda install -c conda-forge deap
from deap import base, creator, tools, algorithms

> 遺伝的アルゴリズム  
> 探索方法を陽に指定する必要がなく、設計ルールの表現のみで探索を実現できる。複雑な制約を避けたり、適応度の評価の難しい問題に対し主観を持ち込んで解決を図る

1. コーディング

+ バイナリコーディング 
+ 順序コーディング 
+ 実数値コーディング 
+ etc

2. 評価

3. 選択

+ エリート選択
+ ランダム選択
+ ルーレット選択 評価点の付け方の影響が大きい
+ トーナメント選択 ランダムに抽出した中から良い個体を選ぶ
+ etc

4a. 交叉
２つの個体からどのように遺伝子を受け継ぐか

+ Ｎ点交叉
+ 一様交叉
+ 順序交叉 一方の個体の残す部分を決め、残りを埋める
+ ブレンド交叉 値の重み付け平均
+ etc

4b. 突然変異

+ 反転
+ 順序入れ替え
+ 値の入れ替え


In [64]:
# ナップザック問題
WEIGHT = 10
items = ((8,10), (7,13), (6,7), (5,4), (4,9), (3,3), (2,3), (1,2))

# 評価関数
def evaluate(ind): # ここでは持っている商品の集合を遺伝子とする
    if len(ind) > len(items):
        return 0,
    w = 0 # 重さ
    v = 0 # 価値
    for idx in ind:
        w += items[idx][0]
        v += items[idx][1]
    v = (v if w <= WEIGHT else 0)
    return v,

# 交叉の定義
def cxSet(ind1, ind2):
    tmp = set(ind1)
    ind1 &= ind2 # 積集合と、
    ind2 ^= tmp # 和集合
    return ind1, ind2

# 突然変異の定義
def mutSet(ind):
    add = random.random() < 0.5
    if add:
        ind.add(random.randrange(len(items)))
    else:
        if len(ind) > 0:
            idx = random.randrange(len(ind))
            ind.remove(tuple(ind)[idx])
    return ind,

In [65]:
import random

creator.create("Fitness", base.Fitness, weights=(1.0,) )
creator.create("Individual", set, fitness = creator.Fitness) # set : 重複を許さない

toolbox = base.Toolbox()

#遺伝子の属性の設定
toolbox.register("attribute", random.randrange, len(items) )
#初期個体の生成
toolbox.register("individual", tools.initRepeat, creator.Indivisual, toolbox.attribute, len(items) )
#初期個体群を作成
toolbox.register("population", tools.initRepeat, list, toolbox.individual )

toolbox.register("evaluate", evaluate)
toolbox.register("mate", cxSet)
toolbox.register("mutate", mutSet)
toolbox.register("select", tools.selTournament, tournsize=3)

In [66]:
hof = tools.ParetoFront()
stats = tools.Statistics(lambda ind: ind.fitness.values) # 評価中に表示する情報
stats.register("avg", np.mean, axis=0)
stats.register("std", np.std, axis=0)
stats.register("min", np.min, axis=0)
stats.register("max", np.max, axis=0)

In [67]:
pop = toolbox.population(n=50)
algorithms.eaSimple(pop, toolbox, cxpb=0.8, mutpb=0.1, ngen=10, stats=stats, halloffame=hof, verbose=True)

gen	nevals	avg 	std 	min 	max 
0  	50    	[0.]	[0.]	[0.]	[0.]
1  	26    	[0.54]	[2.45934951]	[0.]	[13.]
2  	40    	[2.5] 	[4.84252001]	[0.]	[18.]
3  	37    	[6.14]	[6.46532288]	[0.]	[18.]
4  	46    	[4.98]	[5.97826062]	[0.]	[18.]
5  	41    	[5.84]	[5.81501505]	[0.]	[18.]
6  	43    	[5.32]	[6.47592464]	[0.]	[18.]
7  	45    	[5.92]	[5.74400557]	[0.]	[16.]
8  	42    	[5.5] 	[5.96070466]	[0.]	[16.]
9  	43    	[6.82]	[6.6773947] 	[0.]	[18.]
10 	47    	[7.84]	[6.50033845]	[0.]	[18.]


([{1},
  {6, 7},
  {6},
  {1},
  Indivisual(),
  {1, 6},
  {1},
  {6},
  Indivisual(),
  {4, 6, 7},
  {1},
  {7},
  {1, 6},
  {4},
  Indivisual(),
  {0, 1, 6},
  {1, 6},
  {1, 6, 7},
  {1, 6},
  Indivisual(),
  Indivisual(),
  {0, 1, 6, 7},
  {1},
  {2, 6, 7},
  Indivisual(),
  {1, 6},
  Indivisual(),
  {1, 7},
  Indivisual(),
  {1},
  {1},
  {6, 7},
  Indivisual(),
  {1, 6},
  {1, 7},
  Indivisual(),
  {1},
  {6},
  {6},
  {1},
  {1},
  {7},
  {1, 7},
  {6},
  {6},
  {1},
  {6, 7},
  {1, 4},
  {6},
  {1, 7}],
 [{'gen': 0,
   'nevals': 50,
   'avg': array([0.]),
   'std': array([0.]),
   'min': array([0.]),
   'max': array([0.])},
  {'gen': 1,
   'nevals': 26,
   'avg': array([0.54]),
   'std': array([2.45934951]),
   'min': array([0.]),
   'max': array([13.])},
  {'gen': 2,
   'nevals': 40,
   'avg': array([2.5]),
   'std': array([4.84252001]),
   'min': array([0.]),
   'max': array([18.])},
  {'gen': 3,
   'nevals': 37,
   'avg': array([6.14]),
   'std': array([6.46532288]),
   'min'

In [68]:
best_ind = tools.selBest(pop, 1)[0]
best_ind

{1, 6, 7}

In [69]:
evaluate(best_ind)

(18,)